In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

Calculate the self part of the Van Hove correlation function $G_s (r, t)$ at time 0 and $t^*$:
$$
  G_s (r, t_*) =  \frac{1}{N} \sum_{i=1}^{N} \delta [r - |r_i (t_*) - r_i (0)|] 
$$

In [ ]:
def dpart_func(frame0, frame_zv):
    
    dpart = []
    
    for part in frame0['particle']:
        
        if (part in frame_zv['particle'].unique()):
            mask0 = frame0['particle'] == part
            x0 = frame0[mask0]['x'].item()
            y0 = frame0[mask0]['y'].item()

            mask_zv = frame_zv['particle'] == part
            x_zv = frame_zv[mask_zv]['x'].item()
            y_zv = frame_zv[mask_zv]['y'].item()

            dpart.append( abs(np.sqrt(x_zv*x_zv + y_zv*y_zv) - np.sqrt(x0*x0 + y0*y0)) )
    
    
    return dpart
    

In [ ]:
# def Gs_func(dpart, bins, r_max):    

#     # Histogram the radial distances
#     r_values, bin_edges = np.histogram(dpart, bins=bins, range=(0, r_max), density=True)
    
#     # Calculate the midpoints of the bins
#     r_values_mid = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    
    
#     return r_values_mid, r_values

In [ ]:
def Gs_mult_func(r_values, gs_values):
    
    
    return 4 * np.pi * r_values**2 * gs_values

In [ ]:
def aver_func(dpart, r, dr):
    
    G_s = np.zeros_like(r)
    
    N = len(dpart)
    
    for i, r_val in enumerate(r):
        
        count = np.sum((dpart >= r_val) & (dpart < r_val + dr))

        G_s[i] = count / N 
    
    return G_s

In [ ]:
# file_name_prefix = '2_4000_6000'

In [ ]:
def read_data():
    
    file_path = f'data_out/backup/S6T31_t_corrected.csv'
    
    usecols = ['y', 'x', 'frame', 'particle']
    
    data = pd.read_csv(file_path, usecols=usecols)
    
    
    return data

In [ ]:
# write to file Gs

DIR = 'data_out/'

file_r = DIR + 'r.dat'
file_Gs = DIR + 'Gs.dat'

data_out_Gs = pd.DataFrame([])
data_out_Gs.to_csv(file_Gs, mode='w+', index=False, header=False)
    
# create new empty Gs
# with open(DIR + 'Gs.dat', "w+") as file_Gs:
#     pass

In [ ]:
%%time


data = read_data()
# print(data.head())
data_frame0 = data[data['frame'] == 0]
max_frame = data['frame'].values[-1] + 1

# frame_zv = 5900 # t*

r = np.arange(0, 30, 0.0001)
dr = r[1] - r[0]
# write r to file
data_out_dict_r = {"r": r}
data_out_r = pd.DataFrame(data_out_dict_r)
data_out_r.to_csv(file_r, index=False, header=False)


f0 = data[data['frame'] == 0]

Gs_all = []

for frame_zv in range(100, 3800, 50):

    f_zv = data[data['frame'] == frame_zv]


    dpart = dpart_func(f0, f_zv)
    Gs = aver_func(dpart, r, dr)
    # Gs_mult = Gs_mult_func(r, Gs)
    
    Gs_all.extend(Gs)
    
    data_out_dict_Gs = {"Gs": Gs}
    data_out_Gs = pd.DataFrame(data_out_dict_Gs)
    data_out_Gs.to_csv(file_Gs, mode='a', index=False, header=False)
    
    print(f'{frame_zv} --- V')
    
    
    
# r_max = 30
# bins = 300000

# dpart = dpart_func(f0, f_zv)
# # Calculate Gs(r, t)
# r, Gs = Gs_func(dpart, bins, r_max)

# # Calculate 4 * pi * r^2 * Gs(r, t)
# Gs_mult = Gs_mult_func(r, Gs)


In [ ]:
# # write to file Gs

# DIR = f'data_out/Gs/'

# file_Gs = DIR + f'{frame_zv}.dat'

# data_out_dict = {"r[px]": r, "Gs": Gs}

# data_out = pd.DataFrame(data_out_dict)

# data_out.to_csv(file_Gs, index=False)

In [ ]:
# print(len(Gs_all))

In [ ]:
# # write to file Gs

# DIR = 'data_out/'

# file_r = DIR + 'r.dat'
# file_Gs = DIR + 'Gs.dat'

# data_out_dict_r = {"r": r}
# data_out_dict_Gs = {"Gs": Gs_all}

# data_out_r = pd.DataFrame(data_out_dict_r)
# data_out_Gs = pd.DataFrame(data_out_dict_Gs)

# data_out_r.to_csv(file_r, index=False)
# data_out_Gs.to_csv(file_Gs, index=False)

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 6))

# plt.plot(r, Gs_mult, label=r'$4\pi r^2 G_s(r, t)$')

# ax.set(ylabel=r'$G_s$',
#       xlabel='r')

# plt.legend (fontsize=14)

In [ ]:
# # Gaussian approximation of MSD

# def Gauss_MSD(dpart, r):
    
#     delta_r = np.mean( [x*x for x in dpart] )
    
#     G0 = np.zeros_like(r)
    
#     for i, r_val in enumerate(r):

#         G0[i] = (1./(np.pi*delta_r)) * np.exp( -r_val**2 / delta_r )
    
    
#     return G0

In [ ]:
# # convert to microns

# px_to_micron = 0.06905

# r_micron = [i * px_to_micron for i in r]

In [ ]:
# G0 = Gauss_MSD(dpart, r)
# print(G0)

In [ ]:
# Gs_mult = np.zeros_like(r)
# G0_mult = np.zeros_like(r)

# for i, r_val in enumerate(r_micron):
    
#     mult_s = 4*np.pi*r_val**2 #2*np.pi*r_val**2
#     mult_0 = 4*np.pi*r_val**2 #2*np.pi*r_val**2 #2 * np.pi * r_val**2 #4. * np.pi*r_val*r_val
    
#     Gs_mult[i] = mult_s * Gs[i]
#     G0_mult[i] = mult_0 * G0[i]
    
# print(Gs_mult)

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 6))

# # plt.plot(r_micron, Gs_mult, label=r'$4\pi r^2 G_s(r, t)$')
# plt.plot(r, G0, label=r'$4\pi r^2 G_0(r, t)$')

# ax.set(ylabel=r'$G_0, G_s$',
#       xlabel='r')
# # ax.set_xscale('log')
# # ax.set_yscale('log')
# plt.legend (fontsize=14)

In [ ]:
# # write to file G0

# DIR = f'data_out/G0/'

# file_Gs = DIR + f'{frame_zv}.dat'

# data_out_dict = {"r[px]": r, "G0[1/px^2]": G0}

# data_out = pd.DataFrame(data_out_dict)

# data_out.to_csv(file_Gs, index=False)